### Description
This workbook will train a chess AI. It will use a combination of first backwards propagation and then genetic algorithms.  

Imports

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import chess.engine
import chess
import chess.svg
import random
import pandas as pd
import pickle
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import keras.models
import time

Before we create data we have to do some set up.  The plan is to have a 2d input each with a 13 hot vector.  (I chose 13 instead of 12 because I think probably will help teach the format of the board.  Of course you should leave one out in other applications because of the multicollinearity issue).

In [2]:
#input: chess board objects
#output 8,8,13 nparray
def boardToInput(board):
    #I should probably make this a np array but I will have to test if it is faster
    pieces = ['.','r','n','b','q','k','p','P','R','N','B','Q','K']
    #stringify the board
    string = str(board)
    #each col and row has a 13 1 hot vector of the current piece as set in the pieces variable
    arr= np.zeros(shape=(8,8,13))
    row = 0
    col = 0    
    #I know this is stupid we are doing this so many times I don't want to call len(string)
    for indexer in range(127):
        #Skip the spaces
        if string[indexer] == ' ':
            continue
        #This means we are about to move on to a new col
        if string[indexer] == '\n':
            col = 0
            row = row + 1
            continue        
        arr[row][col][pieces.index(string[indexer])] = 1
        col  = col + 1            
    return arr
            

These methods will return np arrays of X and Y.  createData will have Y as the engines eval of the position

# Helper methods.  
These methods do a variety of things most of them are pretty boring but you can look at them.  I will describe them here
### valueToFloat
Stockfish outputs some random integer which can be positive or negative.  Or something like "#+5" which means white has checkmate in 5 moves.  This is really hard to train.  So I convert this into a float [0,1].  Where 0 is the AI found checkmate for black and 1 is checkmate for white.  So is the float is close to 0 the model thinks that black is winning.
### createData
This creates data for the model to train on.  It starts with the start of a game.  It then adds the position to the training data and evaluates the position with stockfish and adds that to the training data.  The next thing it does is make a move.  Usually this is a random move however I found that if stockfish makes the move the model gets better.  However it takes a long time for stockfish to find a move.  The paramater percent_engine_moves dictates what percentage(really it is what percent chance) will the engine make a move vs just a random move.  The two values I have tried are .3 and .5.  .5 took a long time.  Technically I have that data saved on this computer but I havent really used it.  
The other paramater is percent_nonAI_games which is how many of the games should just be played by AI.  I didnt find this to have much of an effect on the model but I only tried low percentages.

### createDataAutoEncoder
I never used this but it creates data for an autoencoder if you want to try that

### getEngineMove
This was pretty slow.  Its fast now.  Gets the best engine move

In [3]:
#Quick method to convert pychess evaluations to floats
def valueToFloat(val):
    #This is emperically the min and the max that I found for each.  Normalized theses
    minEvaluation = -1500
    maxEvaluation = 1500

    string = str(val)
    val = 0
    
    if '#' in string:
        if string[1] == '+':
            return 1
        elif string[1] == '-':
            return 0
    val = float(string)
    val = (val - minEvaluation)/(maxEvaluation-minEvaluation)
    if val < 0:
        val = 0
    if val > 1:
        val = 1
    return val
    #return val
def createData(n,engine,percent_engine_moves=0.3,percent_non_pure_ai_games=1): 
    X = np.zeros((n,8,8,13))
    Y = np.zeros(n)
    isBlack = False
    board = chess.Board()
    for i in tqdm(range(int(n*percent_non_pure_ai_games))):
        isBlack = not isBlack
        if board.outcome() != None: 
            board=chess.Board()
            isBlack = False
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())
        X[i] = (boardToInput(board))
        Y[i] = val
        moves = list(board.legal_moves)
        
        if random.random() > percent_engine_moves:
            move = random.choice(moves)
        else:
            move =getEngineMove(board,engine)
        board.push(move)
    for i in tqdm(range(int(n*percent_non_pure_ai_games),n)):
        isBlack = not isBlack
        if board.outcome() != None: 
            board=chess.Board()
            isBlack = False
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())
        X[i] = (boardToInput(board))
        Y[i] = val
        moves = list(board.legal_moves)
        move =getEngineMove(board,engine)
        board.push(move)
    return X, Y



def createDataAutoEncoder(n,engine): 
    X = np.zeros((n,8,8,13))
    Y = np.zeros(n)
    board = chess.Board()
    for i in tqdm(range(n)):
        if board.outcome() != None: 
            board=chess.Board()
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())
        X[i] = boardToInput(board)
        Y[i] = val
        moves = list(board.legal_moves)
        move = random.choice(moves)
        board.push(move)
    return X, Y


def getMoveEngine(board,moves,isBlack):
    return engine.play(board,limit = chess.engine.Limit(depth=1)).move
    bestEval = -5000
    bestMove = moves[0]
    for move in moves:
        board.push(move)
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())
        if isBlack:
            val =val*-1
        if val > bestEval:
            bestMove = move
            bestEval = val
        board.pop()
    return bestMove
def getEngineMove(board,engine,depth =1):
    return engine.play(board,limit = chess.engine.Limit(depth=depth)).move
#This is old
def getEngineMoveOld(board,engine,isBlack,depth=1):
    bestEval = -5000
    moves = list(board.legal_moves)
    bestMove = moves[0]
    for move in moves:
        board.push(move)
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=depth))['score'].white())
        if isBlack:
            val =val*-1
        if val > bestEval:
            bestMove = move
            bestEval = val
        board.pop()
    return bestMove


# IMPORTANT
Please add your own path to stockfish here

In [4]:
engine = chess.engine.SimpleEngine.popen_uci(".\\stockfish\\stockfish_15.1_win_x64_avx2\\stockfish.exe")

## Training data
This block creates new training data with the params above
Set train_new_model to True if you want to create the data
and save_data to True if you want to save it
Also test_set is false you dont need it because we test the AI on its self!

In [5]:
train_new_model = False
save_data = False
test_set = False


if train_new_model:
    
    training_size = 10000
    testing_size = int(training_size*.2)
    
    Xtr, Ytr = createData(training_size,engine,percent_engine_moves=.5)
    
    if test_set:
        Xte, Yte = createData(testing_size,engine)
        
    if save_data:
        
        import pickle
        dataset = [Xtr,Ytr]
        outputFile = 'tr0500_backup.data'
        fw = open(outputFile, 'wb')
        pickle.dump(dataset, fw)
        fw.close()
        
        
        if test_set:
            dataset = [Xte,Yte]
            outputFile = 'te0500_backup.data'
            fw = open(outputFile, 'wb')
            pickle.dump(dataset, fw)
            fw.close()

In [6]:
#This just displays the data
if train_new_model:
    plt.hist(Ytr,bins=100)
    plt.show()

### Creating the model
To start we will train a basic model with these parameters:
3

In [7]:
if train_new_model:
    model = models.Sequential()
    model.add(layers.Input(shape=(8, 8, 13)))
    #model.add(layers.Conv2D(64, (8, 3), activation='relu', input_shape=(8, 8, 13)))
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation=tf.nn.relu))
    model.add(layers.Dense(128, activation=tf.nn.relu))
    model.add(layers.Dense(64, activation=tf.nn.relu))
    model.add(layers.Dense(128, activation=tf.nn.relu))
    model.add(layers.Dense(128, activation=tf.nn.relu))
    model.add(layers.Dense(128, activation=tf.nn.relu))
    model.add(layers.Dense(64, activation=tf.nn.relu))
    model.add(layers.Dense(64, activation=tf.nn.relu))
    model.add(layers.Dense(1,activation=tf.nn.sigmoid))
    model.summary()

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.callbacks as callbacks

import tensorflow.keras.optimizers as optimizers

if train_new_model:
    #model.compile(optimizer=optimizers.Adam(1e-2), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False))
    model.compile(optimizer=optimizers.Adam(1e-4), loss='Huber')

    #model.summary()
    checkpoint_filepath = '/tmp/checkpoint/'


    model_checkpointing_callback = ModelCheckpoint(
        filepath = checkpoint_filepath,
        save_best_only= True,
    )

    model.fit(Xtr, Ytr,
              batch_size=2048,
              epochs=2,
              verbose=1,
              validation_split=0.1,
                callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                         callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4),model_checkpointing_callback])
    model.save('model_moves50_mixed00.h5')
    model.fit(Xtr, Ytr,
              batch_size=2048,
              epochs=1,
              verbose=1,
              validation_split=0.1,
                callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                         callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4),model_checkpointing_callback])
    model.save('model_moves50_mixed00.h5')

In [9]:
if train_new_model:
    model.save('model_moves03_mixed01_archive.h5')

# Load old model
### This is the best model as of right now

In [10]:
if not train_new_model:
    model = load_model('model_moves30_mixed01_more_training.h5')

# Getting a model move
These methods moves from models with the given params


In [11]:
def getModelMove(board,model,isBlack,depth = 1):
    moves = list(board.legal_moves)
    bestEval = -2
    if isBlack:
        bestEval = bestEval *-1
    bestMove = None
    for move in moves:
        board.push(move)
        if board.is_checkmate():
            board.pop()
            if isBlack:
                return 0,move
            return 1,move
        #val = model(np.array([boardToInput(board)]))
        val = evaluatePosition(board,model,depth-1,not isBlack)
        if isBlack:
            if val < bestEval:
                bestMove = move
                if move == None:
                    print("Oh god")
                bestEval = val
        else:  
            if val > bestEval:
                bestMove = move
                bestEval = val
        board.pop()
    return bestEval,bestMove

In [12]:
#Classic minmax algo
#TODO alpha beta pruning
def evaluatePosition(board,model,depth,isBlack):
    moves = list(board.legal_moves) 
    #print("Depth: "+ str(depth) + " # of moves " + str(len(moves)))
    count = 0
    bestEval = -1
    if isBlack:
        bestEval = 1
    #
    if depth <= 0:
        evaluation = model(np.array([boardToInput(board)]))
        return evaluation
    for move in moves:
        #print("Depth " + str(depth) + " move count " + str(count))
        board.push(move)
        if board.is_checkmate():
            board.pop()
            val = 1
            if isBlack:
                val  = 0
            return val
        else:
            evaluation = evaluatePosition(board,model,depth-1,not isBlack)
        board.pop()
        if isBlack:
            bestEval = min(bestEval,evaluation)
        else:
            bestEval = max(bestEval,evaluation)
        count = count +1
    return bestEval
            

In [67]:
board =chess.Board()
isBlack = False

In [120]:
e,m = getModelMove(board,model,isBlack,depth =2)
print(valueToFloat(engine.analyse(board, chess.engine.Limit(depth=2))['score'].white()))
print(e)
board.push(m)

isBlack = not isBlack
board

0
-2


AttributeError: 'NoneType' object has no attribute 'from_square'

In [112]:

while len(moves) != 0:
    
    print()
    print(isBlack)
    moves = list(board.legal_moves)
    engineMove = getEngineMove(board,engine)
    depth = 1
    if not isBlack:
        depth = 2
    else:
        depth = 1
    evaluation, AImove =  getModelMove(board,model,isBlack,depth=depth)
    move = AImove
    board.push(move)
    print("Model says: " +str(float(evaluation)))
    if isBlack:
        print("Depth 1 eval : "  + str(float(evaluatePosition(board,model,1,isBlack))))
        print("Engine evaluates it at: " + str(valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())))
    else:
        print("Depth 2 eval : "  + str(float(evaluatePosition(board,model,2,isBlack))))
        print("Engine evaluates it at: " + str(valueToFloat(engine.analyse(board, chess.engine.Limit(depth=1))['score'].white())))
    isBlack = not isBlack
    break
board

NameError: name 'moves' is not defined

In [121]:
board =chess.Board()
isBlack = False

In [153]:

if isBlack:
    e,m=getModelMove(board,model,isBlack,depth = 1)
else:
    e,m=getModelMove(board,model,isBlack,depth = 2)
board.push(m)
isBlack = not isBlack
print(float(e))
board

AttributeError: 'NoneType' object has no attribute 'from_square'

In [27]:

def createDataModel(n,model): 
    X = np.zeros((n,8,8,13))
    Y = np.zeros(n)
   
    isBlack = False
    board = chess.Board()

    for i in range(n):
        
        
        isBlack = not isBlack
        if board.outcome() != None: 
            board=chess.Board()
            isBlack = False
        val = valueToFloat(engine.analyse(board, chess.engine.Limit(depth=2))['score'].white())
        
        X[i] = (boardToInput(board))
        Y[i] = val
        
        moves = list(board.legal_moves)
        
        
        e,move = getModelMove(board,model,isBlack)
        board.push(move)
        
    return X, Y


# Training pt 2
This method of training is probably the best.  

In [28]:

for i in tqdm(range(1000)):
    Xtr2,Ytr2 = createDataModel(1000,model)
    model.fit(Xtr2, Ytr2,
              batch_size=100,
              epochs=1,
              verbose=2,
             )
    model.save('model_moves30_mixed01_more_training.h5')


  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

10/10 - 0s - loss: 0.0180 - 175ms/epoch - 17ms/step


  0%|                                                                             | 1/1000 [01:40<27:56:02, 100.66s/it]

10/10 - 0s - loss: 0.0115 - 173ms/epoch - 17ms/step


  0%|▏                                                                            | 2/1000 [03:21<27:53:28, 100.61s/it]

...

10/10 - 0s - loss: 0.0185 - 251ms/epoch - 25ms/step


  7%|█████▍                                                                    | 74/1000 [2:04:11<29:15:34, 113.75s/it]

10/10 - 0s - loss: 0.0161 - 272ms/epoch - 27ms/step


  8%|█████▌                                                                    | 75/1000 [2:06:14<25:56:57, 100.99s/it]


KeyboardInterrupt: 

In [ ]:
board

In [ ]:
def randomize_dense_layers(model,learning_rate,layers=[5,6,7,8,9,10,11]):
    for index in layers:
        n = len(model.layers[index].get_weights()[0])
        m = len(model.layers[index].get_weights()[0][0])
        w= model.layers[index].get_weights()[0] + learning_rate*np.random.normal(size = (n,m))
        n = len(model.layers[index].get_weights()[1])
        b = model.layers[index].get_weights()[1]+learning_rate*np.random.normal(size = n)

        model.layers[index].set_weights([w,b])
def clone_weights(model_unchanged=None,model_changed=None):
    model_changed.set_weights(model.get_weights())
def clone_model(model):
    model_copy = keras.models.clone_model(model)
    model_copy.set_weights(model.get_weights())
    return model_copy

In [ ]:
class genetic_training:
    #TODO layers input
    def __init__(self, n = 20, og_model= None,models=None,depth =1,adaptive_learning_rate =True,learning_rate=1e4):
        self.models = []
        for i in range(n):
            self.models.append(clone_model(og_model))
        self.depth = 1
        self.adaptive_learning_rate = adaptive_learning_rate
        self.learning_rate = learning_rate
    def get_models(self):
        return self.models
    def getModelSplit(self,scoreDict,percentRepoduce=.4):
        sortedModels=dict(sorted(scoreDict.items(),key= lambda x:x[1]))
        return list(sortedModels.keys())[0:int(len(sortedModels.items())*percentRepoduce)],list(sortedModels.keys())[int(len(sortedModels.items())*percentRepoduce):]
    
    def learn(self,scores):
        #good_models,bad_models = self.getModelSplit(scores)
        bad_models,good_models=self.getModelSplit(scores)
        print(scores)
        print(good_models[0])
        '''
        if self.adaptive_learning_rate:
            avgScore = np.mean(np.array(list(scores.values())))
            bestScore = np.min(np.array(list(scores.values())))
            ratio = bestScore/avgScore


            if ratio > self.ALRIthres:
                self.learningRate = min(self.learningRate*self.ALRI,1)
                if verbose > 0:
                    print("Increasing Learning Rate to " + str(self.learningRate))
            elif ratio < self.ALRDthres:
                self.learningRate =max(self.learningRate*self.ALRD,1e-11)
                if verbose > 0:
                    print("Decreasing Learning Rate to " + str(self.learningRate))
        '''

                    
                    
        #this is the actual learning part.  
        for model in self.models:
            if model in bad_models:
                #TODO hyperparmater
                if random.random() > .8:
                    randomize_dense_layers(model,self.learning_rate)
                else:
                    good_model = good_models[int(random.random()*(len(good_models)-1))]
                    clone_weights(model_unchanged=good_model, model_changed=model)
                    randomize_dense_layers(model,self.learning_rate)
                    
    def iteration(self,number_of_rounds=1):
        scores = {}
        total_number_of_games = number_of_rounds* len(self.models)*(len(self.models)-1)
        total_games_played = 0
        avg_time = []
        for i in range(len(self.models)):
            scores[self.models[i]] = 0
        
        for round_i in range(number_of_rounds):
            for model_index in range(len(self.models)):  
                for opp_index in range(len(self.models)):
                    if model_index == opp_index:
                        continue
                    start = time.time()
                    result = play_game(self.models[model_index],self.models[opp_index])
                    scores[self.models[model_index]] =  scores[self.models[model_index]] + result
                    scores[self.models[opp_index]] =  scores[self.models[opp_index]] + (1-result)
                    end = time.time()
                    diff = round(end-start,2)
                    total_games_played = total_games_played +1
                    avg_time.append(diff)
                    print()
                    print("Percent Done: " + str(round(100*total_games_played/total_number_of_games,4))+ "%")
                    print("Last iteration: " + str(diff) + " seconds")
                    print("Avg iteration: " + str(round(np.mean(avg_time),4))+ " seconds")
                    print("EST from avg: " + str(round(np.mean(avg_time)*(total_number_of_games-total_games_played),4)) + " seconds")
        self.learn(scores)
            
        

In [ ]:
def play_game(model_white,model_black,depth=1,verbose=0,board = chess.Board(),number_of_random_moves=2,maxMoves=100):
    isBlack = False
    board = chess.Board()
    moves = list(board.legal_moves)
    move_counter = 0
    for i in range(number_of_random_moves):
        moves = list(board.legal_moves)
        move =random.choice(moves)
        board.push(move)
        isBlack=not isBlack
        
    moves = list(board.legal_moves)    
    while not board.is_game_over():
        if len(moves) == 0:
            return .5
        if isBlack:
            e,move = getModelMove(board,model_black,isBlack,depth=depth)
        else:
            e,move = getModelMove(board,model_white,isBlack,depth=depth)
            
        
        if move == None:
            print(getModelMove(board,model_black,True,depth=depth))
            print(getModelMove(board,model_white,True,depth=depth))
            print(board)
            print(moves)

        board.push(move)
        moves = list(board.legal_moves)
        isBlack = not isBlack
        move_counter = move_counter + 1
        if move_counter > maxMoves:
            return .5   
        if verbose >= 2:
            print(move_counter)
            
            
            
    if board.is_checkmate():
        if isBlack:
            if verbose >= 1:
                print("Black Won")     
            return 0
        else:
            if verbose >= 1:
                print("White Won")
            return 1
    else:
        if verbose >= 1:
            print("Draw")
        return .5
        


In [ ]:
gt.iteration(number_of_rounds=3)

In [ ]:
gt.iteration(number_of_rounds=1)

In [ ]:
gt =genetic_training(n=5,og_model=model)

In [ ]:
board =chess.Board()
isBlack = True
moves = list(board.legal_moves)

In [ ]:
evaluation, AImove =  getModelMove(board,model,isBlack,depth=1)
print(evaluation)
evaluation, AImove =  getModelMove(board,model_copy,isBlack,depth=1)
print(evaluation)

In [ ]:
board =chess.Board()
a,moves =play_game(model2,model,board=board,depth =1)


In [ ]:
print(moves)

In [ ]:
board